<b><h1>CNN (Convolutional Neural Network)</h1></b>
<h2>Computer Vision</h2>

<h2>-> Dog Cat Image Classification</h2>


In [ ]:
import numpy as np
import cv2 # opencv
from PIL import Image # pillow
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
cat_img_path = r"C:\Users\hp\OneDrive\Desktop\day 35\data\images\cats\cat_38.jpg"
cat = Image.open(cat_img_path)

In [ ]:
cat

In [ ]:
type(cat)

In [ ]:
cat_in_pixel = np.array(cat)

In [ ]:
cat_in_pixel

In [ ]:
cat_in_pixel.shape

In [ ]:
import os

cat_images_dir = r"C:\Users\hp\OneDrive\Desktop\day 35\data\images\cats"
dog_images_dir = r"C:\Users\hp\OneDrive\Desktop\day 35\data\images\dogs"

path_dt = {
    "cats": [],
    "dogs": []
}

def collect_image_paths(directory, extensions=('.jpg', '.jpeg', '.png')):
    image_paths = []
    for filename in os.listdir(directory):
        if filename.lower().endswith(extensions):
            image_paths.append(os.path.join(directory, filename))
    return image_paths

path_dt['cats'] = collect_image_paths(cat_images_dir)

path_dt['dogs'] = collect_image_paths(dog_images_dir)

print(path_dt)

len(path_dt['cats'])

In [ ]:
from pathlib import Path

base_directory = r"./Data/images"
for dir_path in os.listdir(base_directory):
  full_path = os.path.join(base_directory, dir_path)
  print(Path(full_path))

In [ ]:
from pathlib import Path

base_directory = r"./Data/images"
for dir_path in os.listdir(base_directory):
  full_path = os.path.join(base_directory, dir_path)
  print(f"{Path(full_path)}")

In [ ]:
Image.open(path_dt['dogs'][10])

In [ ]:
sample_img = cv2.imread(path_dt['dogs'][20])
reduce_img = cv2.resize(sample_img,(224,224))
reduce_img.shape

In [ ]:
# x = []
# y = []

# for i in path_dt['cats']:
#     sample_img = cv2.imread(path_dt['cats'][20])
#     reduce_img = cv2.resize(sample_img,(224,224))
#     x.append(reduce_img)
#     y.append('cats')
# for i in path_dt['dogs']:
#     sample_img = cv2.imread(path_dt['dogs'][20])
#     reduce_img = cv2.resize(sample_img,(224,224))
#     x.append(reduce_img)
#     y.append('dogs')


In [ ]:
x=[]
y=[]

for class_name,paths in path_dt.items():
    for path in paths:
        sample_img = cv2.imread(path)
        reduce_img = cv2.resize(sample_img,(224,224))
        x.append(reduce_img)
        y.append(class_name)

In [ ]:
len(x) , len(y)

In [ ]:
Y = []
for item in y:
    if item == 'cats':
        Y.append(1)
    else:
        Y.append(0)


In [ ]:
set(Y)

In [ ]:
Y

In [ ]:
X = np.array(x)
Y = np.array(Y)

In [ ]:
X

In [ ]:
X.ndim

In [ ]:
type(X) , type(Y)

In [ ]:
X.shape
# 697 images 
# 

In [ ]:
plt.imshow(X[255])
plt.title(y[255])
plt.axis('off')
plt.show()

In [ ]:
# plt.imshow(np.random.randint(0,255,(224,224,3)))
# plt.show()

In [ ]:
X = X/255.0

In [ ]:
X

In [ ]:
x

In [ ]:
plt.imshow(X[255])
plt.title(y[255])
plt.axis('off')
plt.show()

In [ ]:
plt.imshow(x[255])
plt.title(y[255])
plt.axis('off')
plt.show()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
x_train,x_test,y_train,y_test = train_test_split(X,Y,test_size=0.15,random_state=42)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers

In [ ]:
cnn_model = Sequential([
    # Convolutional layers
    layers.Conv2D(64, (3,3), activation='relu', input_shape=(224, 224, 3)),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(40, (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    layers.Conv2D(30, (3,3), activation='relu'),

    #flattening layer
    layers.Flatten(),

    # Dense layers for classification
    layers.Dense(45, activation='relu'),
    layers.Dense(24, activation='relu'),
    
    layers.Dense(1, activation='sigmoid')
])

# Compile the model
cnn_model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Print model summary for a detailed overview
cnn_model.summary()

In [ ]:
# Model Training 
history = cnn_model.fit(x_train,y_train,epochs=10,validation_data=(x_test,y_test))

In [ ]:
import pandas as pd

pd.DataFrame(history.history).plot()

In [ ]:
prediction = cnn_model.predict(x_test)

In [ ]:
pred = (prediction > 0.5).astype(int).ravel()

In [ ]:
pred

In [ ]:
compare_df = pd.DataFrame({"Actual":y_test,"Prediction":pred})
compare_df.head(20)

In [ ]:
from sklearn.metrics import confusion_matrix,classification_report

matrix = confusion_matrix(compare_df['Actual'],compare_df['Prediction'])

sns.heatmap(matrix,annot=True,fmt='d')
plt.title("Confusion Matrix")
plt.axis('off')
plt.show()

In [ ]:
print("Classification Report!")

print(classification_report(compare_df['Actual'],compare_df['Prediction']))

In [ ]:
# Saving the models

import os

os.makedirs('prediction',exist_ok=True)
os.makedirs('models',exist_ok=True)

compare_df['Actual'] = compare_df['Actual'].replace({1: 'cats', 0: 'dogs'})
compare_df['Prediction'] = compare_df['Prediction'].replace({1: 'cats', 0: 'dogs'})


compare_df.to_csv('prediction/predicted.csv',index=False)
cnn_model.save('models/cnn_model.h5')

In [ ]:
compare_df